<a href="https://colab.research.google.com/github/ashtondoane/3d-Four-in-a-Row/blob/master/3D_Four_in_a_Row.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3D Four in a Row engine (using deep search trees and minimax)

- Note that this jupyter notebook was intended for testing the first bots. Finalized code can be found in the Java files.


In [ ]:
import numpy as np

### Creating the bot
1. The two main necessities for a puzzle game engine is the ability to <b>score</b> moves as well as to calculate <b>possible moves</b>
2. The bot will use the <b>minimax</b> function to calculate best future moves, accompanied by 2 other custom functions for <b>scoring</b> and <b> predicting </b> 
3. Multiple scoring functions are implemented in the class, and as such there is a constructor value available to choose which one to use. Multiple can also be employed and averaged.

In [ ]:
class decisionTree:
  def __init__(self, side, e=0.05, scoringAlgorithm): #some unused inputs here such as epsilon. Can be removed in Java
    self.side = side #Side is either white(1) or black(2)
    self.epsilon = e 
    self.scoring = scoringAlgorithm

  def scoreCurrentBoard(self, board):
    if self.scoring == "classic":
      return self.classicScoring(board)

  def classicScoring(self, board): # My original scoring algorithm for a boardstate, as no conclusive game theory exists. Inspired by personal play and trial and error.
    scoreUs = 0
    scoreThem = 0

    if self.haveWon(board, self.side):
      scoreUs = float('inf')
    if self.haveWon(board, 3 - self.side):
      scoreThem = float('inf')

    for i in range(2, 4):
      scoreUs += (i)**1.5 * self.numStreaksRowsCols(board, i, self.side)
      scoreThem += (i)**1.5 * self.numStreaksRowsCols(board, i, 3-self.side)

    scoreUs += 4 * self.numImportantLocationsControlled(board, self.side)
    scoreThem += 4 * self.numImportantLocationsControlled(board, 3-self.side)

    return scoreUs - scoreThem

  def haveWon(self, board, side):
    game = Game(board)
    return game.checkWin(side)


  def numImportantLocationsControlled(self, board, side): #counts the number of outer corners and the center 4 spaces controlled
    count = 0
    points = [[0, 0, 0], [0, 3, 0], [3, 0, 0], [3, 3, 0], [0, 0, 3], [0, 3, 3], [3, 0, 3], [3, 3, 3], [1, 1, 1], [1, 1, 2], [1, 2, 1], [2, 1, 1], [1, 2, 2], [2, 1, 2], [2, 2, 1], [2, 2, 2]]
    for point in points:
      if board[point[0]][point[1]][point[2]] == side:
        count += 1

    return count 
  
  def numStreaksRowsCols(self, board, length, side): #check every row and column and diagonal for the number of pieces you have and if the rest are open.
    count = 0
    otherSide = 3 - side

    i = 0 #jesus christ this is inefficient. Just make it work for now, and you can fix it later to optimize for more looks into the future.
    valid = True
    for stack in board:
      for row in stack:
        valid = True
        for col in row:
          if col == side:
            i += 1
          elif col == (otherSide):
            valid = False
        if i == length and valid:
          count += 1
        i = 0
          
    j = 0
    for stack in range(4):
      for col in range(4):
        valid = True
        for row in range(4):
          if board[stack][row][col] == side:
            j += 1
          elif board[stack][row][col] == (otherSide):
            valid = False
        if j == length and valid:
          count += 1
        j = 0

    k = 0
    for slice in range(4):
      rearrangedBoard = board[0:4, 0:4, slice]
      for col in range(4):
        valid = True
        for row in range(4):
          if rearrangedBoard[row][col] == side:
            k += 1
          elif rearrangedBoard[row][col] == (otherSide):
            valid = False
        if k == length and valid:
          count += 1
        k = 0

    l = 0
    for slice in range(4):
      rearrangedBoard = board[0:4, 0:4, slice]
      for col in range(4):
        valid = True
        for row in range(4):
          if rearrangedBoard[col][row] == side:
            l += 1
          elif rearrangedBoard[col][row] == (otherSide):
            valid = False
        if l == length and valid:
          count += 1
        l = 0


    m = 0
    for slice in range(4):
      rearrangedBoard = board[0:4, slice, 0:4]
      for col in range(4):
        valid = True
        for row in range(4):
          if rearrangedBoard[row][col] == side:
            m += 1
          elif rearrangedBoard[row][col] == (otherSide):
            valid = False
        if m == length and valid:
          count += 1
        m = 0

    n = 0
    for slice in range(4):
      rearrangedBoard = board[0:4, slice, 0:4]
      for col in range(4):
        valid = True
        for row in range(4):
          if rearrangedBoard[col][row] == side:
            n += 1
          elif rearrangedBoard[col][row] == (otherSide):
            valid = False
        if n == length and valid:
          count += 1
        n = 0

    return count/2 #slices are double counting basically everything, as each pair of slices shares one dimension of moves. You could divide count by two at this point to corect 
  
  def numStreaksDiags(self, board, length, side):
    count = 0
    
    i = 0
    j = 0
    for stack in board:
      for diag in range(len(stack)): #More inefficient code
        if stack[diag][diag] == side:
          i += 1
        elif stack[diag][diag] == 3 - side:
          i = 0
          break
        if stack[diag][3-diag] == side:
          j += 1
        elif stack[diag][3 - diag] == 3 - side:
          j = 0
          break
      if i == length: 
        count += 1
        break
      else: 
        i = 0
      if j == length:
        count += 1 
        break
      else: 
        j = 0
  
    m = 0
    n = 0
    for slice in range(4):
      rearrangedBoard = np.array(board[0:4, slice, 0:4])
      for diag in range(len(rearrangedBoard)):
        if rearrangedBoard[diag][diag] == side:
          m += 1
        elif rearrangedBoard[diag][diag] == 3 - side:
          m = 0
          break
        if rearrangedBoard[diag][3-diag] == side:
          n += 1
        elif rearrangedBoard[diag][3 - diag] == 3 - side:
          n = 0
          break
      if m == length: 
        count += 1
        break
      else: 
        m = 0
      if n == length:
        count += 1 
        break
      else: 
        n = 0

    l = 0
    o = 0
    for slice in range(4):
      rearrangedBoard = np.array(board[0:4, 0:4, slice])
      for diag in range(len(rearrangedBoard)):
        if rearrangedBoard[diag][diag] == side:
          l += 1
        elif rearrangedBoard[diag][diag] == 3 - side:
          l = 0
          break
        if rearrangedBoard[diag][3-diag] == side:
          o += 1
        elif rearrangedBoard[diag][3 - diag] == 3 - side:
          o = 0
          break
      if l == length: 
        count += 1
        break
      else: 
        l = 0
      if o == length:
        count += 1 
        break
      else: 
        o = 0

    a = 0
    corners = [[0, 0, 0], [0, 3, 0], [3, 0, 0], [3, 3, 0]]
    corners = np.array(corners)
    slopes = [[1, 1, 1], [1, -1, 1], [-1, 1, 1], [-1, -1, 1]]
    slopes = np.array(slopes)
    for corner in range(4):
      for b in range(4):
        point = corners[corner] + b*slopes[corner]
        if board[point[0]][point[1]][point][2] == side:
          a += 1
        elif board[point[0]][point[1]][point][2] == 3 - side:
          a = 0
          break
      if a == length:
        count += 1
        break
      else: 
        a = 0
    return count

  def calculateBestMove(self, game, side, maximizingPlayer, currentDepth = 1, maxDepth=5): #Recur using minimax algorithm to determine the best move of the possible 16 (or less).
    if type(game) == None and not maximizingPlayer:
      return [float('inf')]
    elif type(game) == None and maximizingPlayer:
      return [-float('inf')]

    if currentDepth == maxDepth:
      return [self.scoreCurrentBoard(game.board)]
    elif self.scoreCurrentBoard(game.board) == abs(float('inf')):
      return [self.scoreCurrentBoard(game.board)]

    x1 = 0
    y1 = 0
    if maximizingPlayer:
      bestValue = -float('inf')
      for i in range(4):
        for j in range(4):
          if game.checkValidPlay(i, j):
            val = self.calculateBestMove(Game(boardstate=game.play(i, j)), side, False, currentDepth+1)[0]
            if val > bestValue:
              bestValue = val
              x1 = i
              y1 = j
            game.remove(i, j)
      return [bestValue, x1, y1]

    else:
      bestValue = float('inf')
      for i in range(4):
        for j in range(4):
          if game.checkValidPlay(i, j):
            val = self.calculateBestMove(Game(boardstate=game.play(i, j)), side, True, currentDepth+1)[0]
            if val < bestValue:
              bestValue = val
              x1 = i
              y1 = j
            game.remove(i, j)
      return [bestValue, x1, y1]


### Creating the board
1. Begin by creating the game space with dimensions 4x4x4
2. Values of 0 on the boardrepresent empty, 1 represent white, and 2 represent black
3. To make the game playable, a game class can be implemented in order to store all the helper functions


In [ ]:
class Game:
  def __init__(self, boardstate=np.array([0]*64)):
    self.board  = boardstate
    self.board = self.board.reshape(4, 4, 4)
    self.encoder = [0, "White", "Black"]
    self.turn = 1

  def play(self, x, y, thisSide = 0):
    if thisSide == 0:
      thisSide = self.turn

    if self.checkValidPlay(x, y):
      self.board[self.getLowestY(x, y)][x][y] = thisSide
      
      if self.turn == 1:
        self.turn = 2
      else: 
        self.turn = 1
      return self.board

    else:
      return None

  def playFormatted(self, x, y): 
    if self.checkValidPlay(x, y):
      self.prevState = self.board
      self.board[self.getLowestY(x, y)][x][y] = self.turn
      
      if self.checkWin(self.turn):
        print(f"{self.encoder[self.turn]} wins!")
      elif self.checkWin(3 - self.turn):
        print(f"{self.encoder[3 - self.turn]} wins!")
      if self.turn == 1:
        self.turn = 2
      else: 
        self.turn = 1
      
      print(self.board)

    else:
      print(f"Invalid play. Try again {self.turn}.")
      print(f"\n{self.board}")

  def remove(self, x, y):
    self.board[self.getLowestY(x, y) - 1][x][y] = 0
    return self.board

  def checkValidPlay(self, x, y):
    if self.board[3][x][y] == 0 and not self.checkWin(self.turn):
      return True
    else:
      return False

  def getLowestY(self, x, y):
    for i in range(3, -1, -1):
      if i == 0 and self.board[i][x][y] == 0:
        return 0
      elif self.board[i][x][y] != 0:
        return i + 1

  def checkWin(self, side): #This code isn't very efficient, but I'm unsure how to optimize it
    tree = decisionTree(side)
    return tree.numStreaksRowsCols(self.board, 4, side) > 0 or tree.numStreaksDiags(self.board, 4, side) > 0


  def reset(self): #reverts the gameboard to its original state
    self.board = np.array([0]*64)
    self.board = self.board.reshape(4, 4, 4)
    print(self.board)

In [ ]:
thisGame = Game()

In [ ]:
tree = decisionTree(1, scoringAlgorithm="classic")

In [ ]:
tree.calculateBestMove(thisGame, tree.side, True, maxDepth=6)[0:3]

[0.0, 0, 0]

In [ ]:
thisGame.play(0, 0)

array([[[1, 2, 0, 0],
        [1, 1, 2, 1],
        [1, 1, 1, 2],
        [2, 1, 2, 2]],

       [[2, 0, 0, 0],
        [0, 2, 1, 0],
        [1, 1, 1, 2],
        [2, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 2, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]])